In [ ]:
import numpy as np # linear algebra
import pandas as pd
import imblearn
df = pd.read_csv("../input/mimic3extracted/mimic-iii-823.csv")
#dropped DOB, and ADMIT_TIME since info in other variables
#dropped diagnosis since to many variables when onehot encoded, partial info encoded in other variables
df = df.drop(['DOB', 'ADMIT_TIME', "DIAGNOSIS", 'HADM_ID', 'SUBJECT_ID'], axis=1)
df = pd.get_dummies(df, columns=['GENDER'], prefix="gender_")
print(df.shape)
df = pd.get_dummies(df, columns=['ADMISSION_TYPE'], prefix="admit_type")
print(df.shape)
df = pd.get_dummies(df, columns=['ADMISSION_LOCATION'], prefix="admit_location")
print(df.shape)
df = pd.get_dummies(df, columns=['INSURANCE'], prefix="insurance")
print(df.shape)
df = pd.get_dummies(df, columns=['LANGUAGE'], prefix="LANGUAGE_")
print(df.shape)
df = pd.get_dummies(df, columns=['RELIGION'], prefix="RELIGION_")
print(df.shape)
df = pd.get_dummies(df, columns=['MARITAL_STATUS'], prefix="MARITAL_STATUS_")
print(df.shape)
df = pd.get_dummies(df, columns=['ETHNICITY'], prefix="ETHNICITY_")
print(df.shape)
df = pd.get_dummies(df, columns=['month_since_last_visit'], prefix="month_since_last_visit")
print(df.shape)


df['year_since_last_visit'] = df['year_since_last_visit'].fillna(df['year_since_last_visit'].max())

In [ ]:
df["HOSPITAL_EXPIRE_FLAG"].value_counts()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
for element in df.select_dtypes(include=['int64']).columns:
    df[[element]] = scaler.fit_transform(df[[element]])

for element in df.select_dtypes(include=['float64']).columns:
    df[[element]] = scaler.fit_transform(df[[element]])

In [ ]:
# dataset SMOTE

from matplotlib import pyplot
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

top_AUC = 0
best_model = None
# define dataset
X = df.drop(["HOSPITAL_EXPIRE_FLAG"], axis=1)
y = df["HOSPITAL_EXPIRE_FLAG"]
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state = 1)
# define pipeline
k = 1
print("random forest results")
while k < 100:
    # evaluate pipeline
    model = RandomForestClassifier(max_depth=k, random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1
    if roc_auc_score(y_test, y_pred) > top_AUC:
        top_AUC = roc_auc_score(y_test, y_pred)
        best_model = model

"""print("SVM results")
k = 1
while k < 8:
    # evaluate pipeline
    model = SVC(degree=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1
    if roc_auc_score(y_test, y_pred) > top_AUC:
        top_AUC = roc_auc_score(y_test, y_pred)
        best_model = model
    
print("GradientBoostingClassifier results")
k = 1
while k < 7:
    # evaluate pipeline
    model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=k, random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1
    if roc_auc_score(y_test, y_pred) > top_AUC:
        top_AUC = roc_auc_score(y_test, y_pred)
        best_model = model
    
print("AdaBoostClassifier results")
k = 15
while k < 10:
    # evaluate pipeline
    model = AdaBoostClassifier(n_estimators=k*25, random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1
    if roc_auc_score(y_test, y_pred) > top_AUC:
        top_AUC = roc_auc_score(y_test, y_pred)
        best_model = model
    
print("Knn results")
k = 3
while k < 10:
    # evaluate pipeline
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1
    if roc_auc_score(y_test, y_pred) > top_AUC:
        top_AUC = roc_auc_score(y_test, y_pred)
        best_model = model
    
print("MLP results")
k = 1
while k < 5:
    # evaluate pipeline
    model = MLPClassifier(solver='adam', hidden_layer_sizes=(50*k,50*k, 50*k), random_state=1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1
    if roc_auc_score(y_test, y_pred) > top_AUC:
        top_AUC = roc_auc_score(y_test, y_pred)
        best_model = model"""

In [ ]:
print(top_AUC)

In [ ]:
!pip install interpret

In [ ]:
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

ebm = ExplainableBoostingClassifier(random_state=1)
ebm.fit(X_train, y_train)
y_pred = ebm.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

ebm_global = ebm.explain_global()
show(ebm_global)

In [ ]:
ebm_local = ebm.explain_local(X_test[:5], y_test[:5])
show(ebm_local)

In [ ]:
!pip install pygam
!pip install flit

In [ ]:
from pygam import LinearGAM, s, f

In [ ]:
from pygam import LinearGAM, s, f

gam = LinearGAM(s(0) + s(1) + f(2)).fit(X_train, y_train)
y_pred = gam.predict(X_test)
y_temp = y_pred > 0.5
print(accuracy_score(y_test, y_temp))
print(roc_auc_score(y_test, y_temp))

In [ ]:
gam = LinearGAM(s(0) + s(1) + s(2) + s(3) + f(4)).fit(X_train, y_train)
y_pred = gam.predict(X_test)
y_temp = y_pred > 0.5
print(accuracy_score(y_test, y_temp))
print(roc_auc_score(y_test, y_temp))

In [ ]:
gam = LinearGAM(s(0) + f(1)).fit(X_train, y_train)
y_pred = gam.predict(X_test)
y_temp = y_pred > 0.5
print(accuracy_score(y_test, y_temp))
print(roc_auc_score(y_test, y_temp))

In [ ]:
gam = LinearGAM(s(0) + s(1) + f(2) + f(3)).fit(X_train, y_train)
y_pred = gam.predict(X_test)
y_temp = y_pred > 0.5
print(accuracy_score(y_test, y_temp))
print(roc_auc_score(y_test, y_temp))

In [ ]:
from pygam import LogisticGAM, s, te

gam = LogisticGAM(s(0) + s(1) + f(2)).fit(X_train, y_train)
y_pred = gam.predict(X_test)
y_temp = y_pred > 0.5
print(accuracy_score(y_test, y_temp))
print(roc_auc_score(y_test, y_temp))

In [ ]:
from pygam import LogisticGAM, s, te

gam = LinearGAM(s(0) + s(1) + f(2)).fit(X_train, y_train)
y_pred = gam.predict(X_test)
y_temp = y_pred > 0.5
print(accuracy_score(y_test, y_temp))
print(roc_auc_score(y_test, y_temp))

In [ ]:
from pygam import PoissonGAM, s, te

gam = PoissonGAM(s(0) + s(1) + f(2)).fit(X_train, y_train)
y_pred = gam.predict(X_test)
y_temp = y_pred > 0.5
print(accuracy_score(y_test, y_temp))
print(roc_auc_score(y_test, y_temp))

In [ ]:
from sklearn.ensemble import VotingClassifier

clf1 = LinearGAM(s(0) + s(1) + f(2)).fit(X_train, y_train)
clf1_prob = LinearGAM(s(0) + s(1) + f(2)).fit(X_train, y_train)
clf2 = RandomForestClassifier(max_depth=50, random_state=1)
clf3 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=6, random_state=0)

eclf_hard = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='hard')
eclf_soft = VotingClassifier(estimators=[('lr', clf1_prob), ('rf', clf2), ('gnb', clf3)],voting='soft')

clf1 = clf1.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)
clf3 = clf3.fit(X_train, y_train)
eclf_hard = eclf_hard.fit(X_train, y_train)
eclf_soft = eclf_soft.fit(X_train, y_train)

print("GAM result:")
y_pred = clf1.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

print("RF result:")
y_pred = clf2.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

print("GBC results")
y_pred = clf3.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

print("hard voting result")
y_pred = eclf_hard.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

print("soft voting result")
y_pred = eclf_soft.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

In [ ]:
from sklearn.ensemble import VotingClassifier

#clf1 = LinearGAM(s(0) + s(1) + s(2) + f(3)).fit(X_train, y_train)
#clf1_prob = LinearGAM(s(0) + s(1) + f(2)).fit(X_train, y_train)
clf2 = RandomForestClassifier(max_depth=50, random_state=1)
clf3 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=6, random_state=0)
clf4 = MLPClassifier(solver='adam', hidden_layer_sizes=(150,150, 150), random_state=1)
clf5 = ExplainableBoostingClassifier(random_state=1)

eclf_hard = VotingClassifier(estimators=[('rf', clf2), ('gnb', clf3), ("ebc", clf5)],voting='hard')
eclf_soft = VotingClassifier(estimators=[('rf', clf2), ('gnb', clf3), ("ebc", clf5)],voting='soft')


eclf_hard = eclf_hard.fit(X_train, y_train)
eclf_soft = eclf_soft.fit(X_train, y_train)

print("hard voting result")
y_pred = eclf_hard.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

print("soft voting result")
y_pred = eclf_soft.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

In [ ]:
from sklearn.ensemble import VotingClassifier

clf1 = RandomForestClassifier(max_depth=57, random_state=0)
clf2 = RandomForestClassifier(max_depth=68, random_state=0)
clf3 = RandomForestClassifier(max_depth=66, random_state=0)

eclf_hard = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='hard')
eclf_soft = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='soft')

clf1 = clf1.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)
clf3 = clf3.fit(X_train, y_train)
eclf_hard = eclf_hard.fit(X_train, y_train)
eclf_soft = eclf_soft.fit(X_train, y_train)

print("SVM result:")
y_pred = clf1.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

print("RF result:")
y_pred = clf2.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

print("MLP results")
y_pred = clf3.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

print("hard voting result")
y_pred = eclf_hard.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

print("soft voting result")
y_pred = eclf_soft.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

In [ ]:
feature_importances = pd.DataFrame(best_model.feature_importances_,index = list(X.columns), columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

top_5 = feature_importances.head(5).index
top_10 = feature_importances.head(10).index
top_15 = feature_importances.head(15).index
top_20 = feature_importances.head(20).index
top_25 = feature_importances.head(25).index
top_30 = feature_importances.head(30).index
top_35 = feature_importances.head(35).index
top_40 = feature_importances.head(40).index
top_45 = feature_importances.head(45).index
top_50 = feature_importances.head(50).index
top_features = [top_5, top_10, top_15, top_20, top_25,top_30, top_35,top_40, top_45, top_50]

In [ ]:
top_AUC_rf = 0
best_model_rf = None

top_AUC_svm = 0
best_model_svm = None

top_AUC_gbc = 0
best_model_gbc = None

top_AUC_knn = 0
best_model_knn = None

top_AUC_ada = 0
best_model_ada = None

top_AUC_mlp = 0
best_model_mlp = None

for element in top_features:
    # define dataset
    X = df[element]
    y = df["HOSPITAL_EXPIRE_FLAG"]
    sm = SMOTE(random_state=42)
    X_res, y_res = sm.fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state = 1)
    # define pipeline
    """k = 1
    while k < len(element) and k < 35:
    # define pipeline
        print("random forest results for:" + str(len(element)))
        # evaluate pipeline
        model = RandomForestClassifier(max_depth=k, random_state=0)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print(k)
        print(accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        k = k + 1
        if roc_auc_score(y_test, y_pred) > top_AUC_rf:
            top_AUC_rf = roc_auc_score(y_test, y_pred)
            best_model_rf = model
            
    print("SVM results")
    k = 1
    while k < 8:
        # evaluate pipeline
        model = SVC(degree=k)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print(k)
        print(accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        k = k + 1
        if roc_auc_score(y_test, y_pred) > top_AUC_svm:
            top_AUC_svm = roc_auc_score(y_test, y_pred)
            best_model_svm = model

    print("GradientBoostingClassifier results")
    k = 1
    while k < 7:
        # evaluate pipeline
        model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=k, random_state=0)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print(k)
        print(accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        k = k + 1
        if roc_auc_score(y_test, y_pred) > top_AUC_gbc:
            top_AUC_gbc = roc_auc_score(y_test, y_pred)
            best_model_gbc = model

    print("AdaBoostClassifier results")
    k = 15
    while k < 20:
        # evaluate pipeline
        model = AdaBoostClassifier(n_estimators=k*25, random_state=0)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print(k)
        print(accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        k = k + 1
        if roc_auc_score(y_test, y_pred) > top_AUC_ada:
            top_AUC_ada = roc_auc_score(y_test, y_pred)
            best_model_ada = model

    print("Knn results")
    k = 3
    while k < 10:
        # evaluate pipeline
        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print(k)
        print(accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        k = k + 1
        if roc_auc_score(y_test, y_pred) > top_AUC_knn:
            top_AUC_knn = roc_auc_score(y_test, y_pred)
            best_model_knn = model

    print("MLP results")
    k = 1
    while k < 8:
        # evaluate pipeline
        model = MLPClassifier(solver='adam', hidden_layer_sizes=(25*k,25*k, 25*k), random_state=1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print(k)
        print(accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        k = k + 1
        if roc_auc_score(y_test, y_pred) > top_AUC_mlp:
            top_AUC_mlp = roc_auc_score(y_test, y_pred)
            best_model_mlp = model
    """
    from interpret import set_visualize_provider
    from interpret.provider import InlineProvider
    set_visualize_provider(InlineProvider())

    from interpret.glassbox import ExplainableBoostingClassifier
    from interpret import show

    ebm = ExplainableBoostingClassifier(random_state=1)
    ebm.fit(X_train, y_train)
    y_pred = ebm.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    
    """try:
        gam = LinearGAM(s(0) + s(1) + f(2)).fit(X_train, y_train)
        y_pred = gam.predict(X_test)
        y_temp = y_pred > 0.5
        print(accuracy_score(y_test, y_temp))
        print(roc_auc_score(y_test, y_temp))
    except:
        gam = LinearGAM(s(0) + s(1) + s(2) + f(3)).fit(X_train, y_train)
        y_pred = gam.predict(X_test)
        y_temp = y_pred > 0.5
        print(accuracy_score(y_test, y_temp))
        print(roc_auc_score(y_test, y_temp))"""
    print("done:" + str(element))

In [ ]:
for element in top_features:
    # define dataset
    X = df[element]
    X = df[top_15]
    y = df["HOSPITAL_EXPIRE_FLAG"]
    sm = SMOTE(random_state=42)
    X_res, y_res = sm.fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2)
    # define pipeline
    ebm1 = ExplainableBoostingClassifier(random_state=1)


    ebm2 = ExplainableBoostingClassifier(random_state=2)


    ebm3 = ExplainableBoostingClassifier(random_state=3)




    from sklearn.ensemble import VotingClassifier

    eclf_hard = VotingClassifier(estimators=[('rf', ebm1), ('gnb', ebm2), ("ebc", ebm3)],voting='hard')
    eclf_soft = VotingClassifier(estimators=[('rf', ebm1), ('gnb', ebm2), ("ebc", ebm3)],voting='soft')


    eclf_hard = eclf_hard.fit(X_train, y_train)
    eclf_soft = eclf_soft.fit(X_train, y_train)

    print("hard voting result")
    y_pred = eclf_hard.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))

    print("soft voting result")
    y_pred = eclf_soft.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))

    ebm1.fit(X_train, y_train)
    y_pred = ebm1.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))

    ebm2.fit(X_train, y_train)
    y_pred = ebm2.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))

    ebm3.fit(X_train, y_train)
    y_pred = ebm3.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))

In [ ]:
feature_importances = pd.DataFrame(best_model.feature_importances_,index = list(feature_importances.head(40).index), columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances['importance'].to_numpy().size)
print(feature_importances)

In [ ]:
df[top_15]                                      

In [ ]:
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

X = df[element]
X = df[top_15]
y = df["HOSPITAL_EXPIRE_FLAG"]
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state = 1)
    # define pipeline
ebm1 = ExplainableBoostingClassifier(random_state=1)
ebm1.fit(X_train, y_train)
y_pred = ebm1.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))
ebm1_global = ebm1.explain_global()
show(ebm1_global)

In [ ]:
X_test[:50]

In [ ]:
ebm1_local = ebm1.explain_local(X_test[:50], y_test[:50])
show(ebm1_local)

In [ ]:
# dataset not changes

from matplotlib import pyplot
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
# define dataset
X = df.drop(["HOSPITAL_EXPIRE_FLAG"], axis=1)
y = df["HOSPITAL_EXPIRE_FLAG"]
#sm = SMOTE(random_state=42)
#X_res, y_res = sm.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# define pipeline
k = 1
print("random forest results")
while k < 35:
    # evaluate pipeline
    model = RandomForestClassifier(max_depth=k, random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1

print("SVM results")
k = 1
while k < 8:
    # evaluate pipeline
    model = SVC(degree=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1
    
print("GradientBoostingClassifier results")
k = 1
while k < 7:
    # evaluate pipeline
    model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=k, random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1
    
print("AdaBoostClassifier results")
k = 1
while k < 7:
    # evaluate pipeline
    model = AdaBoostClassifier(n_estimators=k*25, random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1
    
print("Knn results")
k = 3
while k < 10:
    # evaluate pipeline
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1
    
print("MLP results")
k = 1
while k < 10:
    # evaluate pipeline
    model = MLPClassifier(solver='adam', hidden_layer_sizes=(10*k,10*k, 10*k), random_state=1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1

In [ ]:
# dataset randomundersampling

from matplotlib import pyplot
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
# define dataset
X = df.drop(["HOSPITAL_EXPIRE_FLAG"], axis=1)
y = df["HOSPITAL_EXPIRE_FLAG"]
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2)
# define pipeline
k = 1
print("random forest results")
while k < 35:
    # evaluate pipeline
    model = RandomForestClassifier(max_depth=k, random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1

print("SVM results")
k = 1
while k < 8:
    # evaluate pipeline
    model = SVC(degree=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1
    
print("GradientBoostingClassifier results")
k = 1
while k < 7:
    # evaluate pipeline
    model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=k, random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1
    
print("AdaBoostClassifier results")
k = 1
while k < 7:
    # evaluate pipeline
    model = AdaBoostClassifier(n_estimators=k*25, random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1
    
print("Knn results")
k = 3
while k < 10:
    # evaluate pipeline
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1
    
print("MLP results")
k = 1
while k < 10:
    # evaluate pipeline
    model = MLPClassifier(solver='adam', hidden_layer_sizes=(10*k,10*k, 10*k), random_state=1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(k)
    print(accuracy_score(y_test, y_pred))
    print(roc_auc_score(y_test, y_pred))
    k = k + 1